In [1]:
import sys
sys.path.append("..")

In [2]:
import numpy as np
import logging
import pickle

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import RobustScaler
from sklearn.utils import check_random_state

from recnn.preprocessing import permute_by_pt
from recnn.preprocessing import rotate
from recnn.recnn import grnn_transform_gated
from recnn.recnn import grnn_predict_gated

%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8,8)

In [3]:
filename_train = "../data/w-vs-qcd/kt-train.pickle"
filename_test = "../data/w-vs-qcd/kt-test.pickle"

rng = check_random_state(1)

# Make training data
print("Loading training data...")

fd = open(filename_train, "rb")
X, y = pickle.load(fd)
fd.close()
y = np.array(y)

print("\tfilename = %s" % filename_train)
print("\tX size = %d" % len(X))
print("\ty size = %d" % len(y))

# Preprocessing 
print("Preprocessing...")
X = [rotate(permute_by_pt(jet)) for jet in X]
tf = RobustScaler().fit(np.vstack([jet["content"] for jet in X]))

for jet in X:
    jet["content"] = tf.transform(jet["content"])

# Make test data 
print("Loading test data...")

fd = open(filename_test, "rb")
X, y = pickle.load(fd)
fd.close()
y = np.array(y)

print("\tfilename = %s" % filename_test)
print("\tX size = %d" % len(X))
print("\ty size = %d" % len(y))

# Preprocessing 
print("Preprocessing...")
X = [rotate(permute_by_pt(jet)) for jet in X]

for jet in X:
    jet["content"] = tf.transform(jet["content"])

Loading training data...
	filename = ../data/w-vs-qcd/kt-train.pickle
	X size = 80000
	y size = 80000
Preprocessing...
Loading test data...
	filename = ../data/w-vs-qcd/kt-test.pickle
	X size = 20000
	y size = 20000
Preprocessing...


In [4]:
# Loading model 
print("Loading model...")

filename_model = "../models/step_size+decay/w-simple-0.001-0.9.pickle"
fd = open(filename_model, "rb")
params = pickle.load(fd)
fd.close()

print("\tfilename = %s" % filename_model)

Loading model...
	filename = ../models/step_size+decay/w-simple-0.001-0.9.pickle


In [5]:
from recnn.recnn import grnn_predict_simple
from sklearn.metrics import roc_auc_score
roc_auc_score(y, grnn_predict_simple(params, X))

0.88219467000000007

In [28]:
import autograd.numpy as np
from recnn.recnn import relu, sigmoid

def grnn_predict(v, node_id, params, jet):   
    tree = jet["tree"]
    
    content = jet["content"]
    content = np.vstack([content[:node_id], 
                         v.reshape(1, -1),  # hook in new content
                         content[node_id+1:]])
    
    def _rec(node_id):
        u_k = relu(np.dot(params["W_u"], content[node_id].T).T + 
                   params["b_u"]).reshape(1, -1)
        
        if tree[node_id, 0] == -1:
            return u_k 
        
        else:
            h_L = _rec(tree[node_id, 0])
            h_R = _rec(tree[node_id, 1])
            h = relu(np.dot(params["W_h"],
                            np.hstack((h_L, h_R, u_k)).T).T +
                     params["b_h"])
            return h
        
    h = _rec(jet["root_id"])
    
    h = relu(np.dot(params["W_clf"][0], h.T).T + params["b_clf"][0])
    h = relu(np.dot(params["W_clf"][1], h.T).T + params["b_clf"][1])
    h = sigmoid(np.dot(params["W_clf"][2], h.T).T + params["b_clf"][2])

    return h.ravel()

In [29]:
# double check this rewritten function gives same results

for jet_id in np.random.permutation(len(X))[:10]:
    jet = X[jet_id]
    j = np.random.randint(len(jet["tree"]))
    print(grnn_predict(jet["content"][j], j, params, jet))
    print(grnn_predict_simple(params, [jet]))

[ 0.0958655]
[ 0.0958655]
[ 0.2497271]
[ 0.2497271]
[ 0.96485677]
[ 0.96485677]
[ 0.19484276]
[ 0.19484276]
[ 0.46790763]
[ 0.46790763]
[ 0.18409799]
[ 0.18409799]
[ 0.96548445]
[ 0.96548445]
[ 0.12093088]
[ 0.12093088]
[ 0.22971709]
[ 0.22971709]
[ 0.26538199]
[ 0.26538199]


In [80]:
import autograd as ag
df = ag.grad(grnn_predict)

jet = X[123]
node_id = 50
v = jet["content"][node_id].copy()

for i in range(100):
    print(grnn_predict(v, node_id, params, jet))
    v += df(v, node_id, params, jet)   # negative step to make it more background like and vice-versa


[ 0.37480144]
[ 0.37521856]
[ 0.37563605]
[ 0.3760539]
[ 0.37647213]
[ 0.37689073]
[ 0.37730969]
[ 0.37772902]
[ 0.37814872]
[ 0.37856878]
[ 0.37898921]
[ 0.37941]
[ 0.37985504]
[ 0.3806107]
[ 0.38136752]
[ 0.38212549]
[ 0.38288462]
[ 0.38364489]
[ 0.38440631]
[ 0.38516886]
[ 0.38593254]
[ 0.38669734]
[ 0.38746327]
[ 0.38823032]
[ 0.38899847]
[ 0.38976773]
[ 0.39052078]
[ 0.39121537]
[ 0.39191085]
[ 0.39260721]
[ 0.3931951]
[ 0.39365516]
[ 0.39388979]
[ 0.39412452]
[ 0.39435934]
[ 0.39459427]
[ 0.39482929]
[ 0.3950644]
[ 0.39529962]
[ 0.39553493]
[ 0.39577034]
[ 0.39600585]
[ 0.39624145]
[ 0.39647715]
[ 0.39671294]
[ 0.39694884]
[ 0.39718482]
[ 0.39742091]
[ 0.39765708]
[ 0.39789336]
[ 0.39816124]
[ 0.39848976]
[ 0.39881846]
[ 0.39914735]
[ 0.39947642]
[ 0.39980567]
[ 0.4001351]
[ 0.40046525]
[ 0.4007975]
[ 0.40112993]
[ 0.40145575]
[ 0.40176891]
[ 0.40208222]
[ 0.40239569]
[ 0.40270933]
[ 0.40302312]
[ 0.40333708]
[ 0.40365119]
[ 0.40396546]
[ 0.40427826]
[ 0.40458926]
[ 0.40490041]
[